In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need these
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/recommender/movielens/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import tensorflow as tf
print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())
import pprint
import logging

from pathlib import Path

TensorFlow Version 1.14.0
GPU Enabled: True


In [0]:
# stream data from text files
def gen_fn(f_path):
  movietype2idx = {}
  with open('../vocab/movie_types.txt') as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      movietype2idx[line] = i
  
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      (user_id, user_gender, user_age, user_job, movie_id, movie_types, movie_title, score) = line.split('\t')
      
      movie_types_ = [0] * len(movietype2idx)
      for movie_type in movie_types.split():
        movie_types_[movietype2idx[movie_type]] = 1
        
      movie_title = movie_title.split()
      yield (user_id, user_age, user_job, user_gender, movie_id, movie_types_, movie_title), score


def input_fn(mode, params):
  _shapes = (([], [], [], [], [], [18], [None]), [])
  _types = ((tf.string, tf.string, tf.string, tf.string, tf.string, tf.int32, tf.string), tf.float32)
  _pads = (('-1', '-1', '-1', '-1', '-1', -1, '<pad>'), 0.)
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    ds = tf.data.Dataset.from_generator(
      lambda: gen_fn('../data/train.txt'),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.repeat()
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    
  if mode == tf.estimator.ModeKeys.EVAL:
    ds = tf.data.Dataset.from_generator(
      lambda: gen_fn('../data/test.txt'),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  return ds

In [0]:
def model_fn(features, labels, mode, params):
  # Receive inputs
  user_id, user_age, user_job, user_gender, movie_id, movie_types, movie_title = features
  
  
  # Flag for Dropout / Batch Norm
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  
  # Word Indexing
  lookup_user_id = tf.contrib.lookup.index_table_from_file(
    '../vocab/user_id.txt', num_oov_buckets=1)
  
  lookup_user_age = tf.contrib.lookup.index_table_from_file(
    '../vocab/user_age.txt', num_oov_buckets=1)
  
  lookup_user_job = tf.contrib.lookup.index_table_from_file(
    '../vocab/user_job.txt', num_oov_buckets=1)
  
  lookup_user_gender = tf.contrib.lookup.index_table_from_file(
    '../vocab/user_gender.txt', num_oov_buckets=1)
  
  lookup_movie_id = tf.contrib.lookup.index_table_from_file(
    '../vocab/movie_id.txt', num_oov_buckets=1)
  
  lookup_movie_title = tf.contrib.lookup.index_table_from_file(
    '../vocab/movie_title.txt', num_oov_buckets=1)
  
  user_id = lookup_user_id.lookup(user_id)
  
  user_age = lookup_user_age.lookup(user_age)
  
  user_job = lookup_user_job.lookup(user_job)
  
  user_gender = lookup_user_gender.lookup(user_gender)
  
  movie_id = lookup_movie_id.lookup(movie_id)
  
  movie_title = lookup_movie_title.lookup(movie_title)
  
  
  # Embedding
  user_id = tf.contrib.layers.embed_sequence(
    ids = user_id,
    vocab_size = params['user_id_size'] + 1,
    embed_dim = params['large_embed_dim'],
    scope='user_id')
  
  user_age = tf.contrib.layers.embed_sequence(
    ids = user_age,
    vocab_size = params['user_age_size'] + 1,
    embed_dim = params['small_embed_dim'],
    scope='user_age')
  
  user_job = tf.contrib.layers.embed_sequence(
    ids = user_job,
    vocab_size = params['user_job_size'] + 1,
    embed_dim = params['small_embed_dim'],
    scope='user_job')
  
  user_gender = tf.contrib.layers.embed_sequence(
    ids = user_gender,
    vocab_size = params['user_gender_size'],
    embed_dim = params['small_embed_dim'],
    scope='user_gender')
  
  movie_id = tf.contrib.layers.embed_sequence(
    ids = movie_id,
    vocab_size = params['movie_id_size'] + 1,
    embed_dim = params['large_embed_dim'],
    scope='movie_id')
  
  movie_types = tf.to_float(movie_types)
  
  movie_title = tf.contrib.layers.embed_sequence(
    ids = movie_title,
    vocab_size = params['movie_title_size'] + 1,
    embed_dim = params['large_embed_dim'],
    scope='movie_title')
  
  
  # User Network
  user_feature = tf.concat((user_id, user_age, user_job, user_gender), -1)
  
  user_feature = tf.layers.dropout(user_feature, params['dropout_rate'], training=is_training)
  
  user_feature = tf.layers.dense(user_feature,
                                 params['hidden_dim'],
                                 params['activation'],
                                 name='user_feature/fc')
  
  
  # Movie Network
  movie_title = tf.layers.dropout(movie_title, params['dropout_rate'], training=is_training)
  
  movie_title = tf.reduce_max(tf.layers.conv1d(movie_title,
                                               filters=params['large_embed_dim'],
                                               kernel_size=params['kernel_size'],
                                               activation=params['activation'],
                                               name='movie_feature/conv1d'), axis=1)
  
  movie_feature = tf.concat((movie_id, movie_types, movie_title), -1)
  
  movie_feature = tf.layers.dropout(movie_feature, params['dropout_rate'], training=is_training)
  
  movie_feature = tf.layers.dense(movie_feature,
                                  params['hidden_dim'],
                                  params['activation'],
                                  name='movie_feature/fc')
  
  
  # Aggregation
  scores = tf.concat([tf.abs(user_feature - movie_feature),
                      user_feature * movie_feature,
                      user_feature,
                      movie_feature], -1)
  scores = tf.layers.dropout(scores, params['dropout_rate'], training=is_training)
  scores = tf.layers.dense(scores, params['hidden_dim'], params['activation'])
  scores = tf.layers.dropout(scores, params['dropout_rate'], training=is_training)
  scores = tf.layers.dense(scores, params['hidden_dim'], params['activation'])
  scores = tf.layers.dense(scores, 1)
  scores = tf.squeeze(scores, -1)
  predictions = 5. * tf.sigmoid(scores)
  
  
  if labels is not None:
    labels = (labels + 5.) / 2.
    loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
      labels=labels/5., logits=scores))
  
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
    
    global_step=tf.train.get_or_create_global_step()
    decay_lr = tf.train.exponential_decay(
      params['lr'], global_step, 1000, .96)
    
    optim = tf.train.AdamOptimizer(decay_lr)
    train_op = optim.minimize(
      loss_op, global_step=tf.train.get_or_create_global_step())
    
    hook = tf.train.LoggingTensorHook({'lr': decay_lr}, every_n_iter=100)
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss_op,
                                      train_op=train_op,
                                      training_hooks=[hook],)
  
  
  if mode == tf.estimator.ModeKeys.EVAL:
    mae_op = tf.metrics.mean_absolute_error(labels=labels,
                                            predictions=predictions)
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss_op,
                                      eval_metric_ops={'mae': mae_op})

In [0]:
params = {
    'log_path': '../log/dnn.txt',
    'model_dir': '../model/dnn',
    'user_id_size': 6040,
    'user_age_size': 7,
    'user_job_size': 21,
    'user_gender_size': 2,
    'movie_id_size': 3691,
    'movie_title_size': 3761,
    'small_embed_dim': 30,
    'large_embed_dim': 200,
    'hidden_dim': 200,
    'activation': tf.nn.elu,
    'kernel_size': 3,
    'dropout_rate': 0.2,
    'lr': 3e-4,
    'num_patience': 7,
    'buffer_size': int(6E5),
    'batch_size': 256,
}

In [6]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
config = tf.estimator.RunConfig(
  save_checkpoints_steps=params['buffer_size']//params['batch_size'],
  keep_checkpoint_max=params['num_patience']+2,)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=config,
  params=params)

# This hook early-stops model if testing accuracy is not improved
hook = tf.estimator.experimental.stop_if_no_decrease_hook(
  estimator=estimator,
  metric_name='mae',
  max_steps_without_decrease=params['num_patience']*params['buffer_size']//params['batch_size'],
  run_every_secs=None,
  run_every_steps=params['buffer_size']//params['batch_size'])

# Train on training data and Evaluate on testing data
train_spec = tf.estimator.TrainSpec(
  input_fn=lambda: input_fn(mode=tf.estimator.ModeKeys.TRAIN,
                            params=params),
  hooks=[hook])

eval_spec = tf.estimator.EvalSpec(
  input_fn=lambda: input_fn(mode=tf.estimator.ModeKeys.EVAL,
                            params=params),
  steps=None,
  throttle_secs=10,)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

I0709 01:54:44.455173 139621405767552 estimator.py:209] Using config: {'_model_dir': '../model/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2343, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 9, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbdb0b3f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
I0709 01:54:44.458442 139621405767552 estimator_training.py:186] Not using Distribute Coordinator

Reading ../data/train.txt


I0709 01:56:42.303075 139621405767552 basic_session_run_hooks.py:262] loss = 0.6957916, step = 0
I0709 01:56:42.304564 139621405767552 basic_session_run_hooks.py:262] lr = 0.0003
I0709 01:56:48.729285 139621405767552 basic_session_run_hooks.py:692] global_step/sec: 15.5585
I0709 01:56:48.733696 139621405767552 basic_session_run_hooks.py:260] loss = 0.570103, step = 100 (6.431 sec)
I0709 01:56:48.736675 139621405767552 basic_session_run_hooks.py:260] lr = 0.00029877786 (6.432 sec)
I0709 01:56:54.608517 139621405767552 basic_session_run_hooks.py:692] global_step/sec: 17.0091
I0709 01:56:54.610535 139621405767552 basic_session_run_hooks.py:260] loss = 0.56949306, step = 200 (5.877 sec)
I0709 01:56:54.613451 139621405767552 basic_session_run_hooks.py:260] lr = 0.00029756065 (5.877 sec)
I0709 01:57:00.443798 139621405767552 basic_session_run_hooks.py:692] global_step/sec: 17.1371
I0709 01:57:00.447001 139621405767552 basic_session_run_hooks.py:260] loss = 0.54932284, step = 300 (5.836 sec)


Reading ../data/train.txt


I0709 01:57:53.259103 139621405767552 basic_session_run_hooks.py:692] global_step/sec: 16.9598
I0709 01:57:53.262758 139621405767552 basic_session_run_hooks.py:260] loss = 0.5385315, step = 1200 (5.897 sec)
I0709 01:57:53.265128 139621405767552 basic_session_run_hooks.py:260] lr = 0.00028565823 (5.895 sec)
I0709 01:57:59.110134 139621405767552 basic_session_run_hooks.py:692] global_step/sec: 17.091
I0709 01:57:59.115243 139621405767552 basic_session_run_hooks.py:260] loss = 0.5532668, step = 1300 (5.852 sec)
I0709 01:57:59.117090 139621405767552 basic_session_run_hooks.py:260] lr = 0.00028449448 (5.852 sec)
I0709 01:58:04.988655 139621405767552 basic_session_run_hooks.py:692] global_step/sec: 17.0111
I0709 01:58:04.990632 139621405767552 basic_session_run_hooks.py:260] loss = 0.581186, step = 1400 (5.875 sec)
I0709 01:58:04.993137 139621405767552 basic_session_run_hooks.py:260] lr = 0.0002833355 (5.876 sec)
I0709 01:58:10.930296 139621405767552 basic_session_run_hooks.py:692] global_st

...

Reading ../data/test.txt


I0709 02:59:22.371781 139621405767552 evaluation.py:275] Finished evaluation at 2019-07-09-02:59:22
I0709 02:59:22.373052 139621405767552 estimator.py:2039] Saving dict for global step 53889: global_step = 53889, loss = 0.54498446, mae = 0.66572577
I0709 02:59:22.380182 139621405767552 estimator.py:2099] Saving 'checkpoint_path' summary for global step 53889: ../model/dnn/model.ckpt-53889
I0709 02:59:22.456585 139621405767552 early_stopping.py:415] No decrease in metric "mae" for 18744 steps, which is greater than or equal to max steps (16406) configured for early stopping.
I0709 02:59:22.457662 139621405767552 early_stopping.py:485] Requesting early stopping at global step 53889
I0709 02:59:22.495007 139621405767552 basic_session_run_hooks.py:606] Saving checkpoints for 53890 into ../model/dnn/model.ckpt.
I0709 02:59:22.796017 139621405767552 estimator.py:1145] Calling model_fn.
I0709 02:59:23.018811 139621405767552 estimator.py:1147] Done calling model_fn.
I0709 02:59:23.041270 13962

Reading ../data/test.txt


I0709 02:59:44.820681 139621405767552 evaluation.py:275] Finished evaluation at 2019-07-09-02:59:44
I0709 02:59:44.827504 139621405767552 estimator.py:2039] Saving dict for global step 53890: global_step = 53890, loss = 0.5449746, mae = 0.66575235
I0709 02:59:44.829806 139621405767552 estimator.py:2099] Saving 'checkpoint_path' summary for global step 53890: ../model/dnn/model.ckpt-53890
I0709 02:59:48.564234 139621405767552 estimator.py:368] Loss for final step: 0.55210155.


({'global_step': 53890, 'loss': 0.5449746, 'mae': 0.66575235}, [])